# 모듈 import 및 token, timezone 설정

In [2]:
import datetime
import json
import requests
import os
import tqdm
import pymongo

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
# bearer_token = os.environ.get("BEARER_TOKEN")
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAACa4YwEAAAAAeM368wj9N3QfGf2t6EoGL2amj8Y%3DvrVlm8AlRWEDnnhD2XtW1rE35Wav9eAeTxhkQo8C0yFRwHvEzm"
KST = datetime.timezone(datetime.timedelta(hours=9))

In [3]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {BEARER_TOKEN}"
    r.headers["User-Agent"] = "v2SampledStreamPython"
    return r

# 수집

In [4]:
url = "https://api.twitter.com/2/tweets/sample/stream"
response = requests.request("GET", url, auth=bearer_oauth, stream=True)

In [5]:
mongo = pymongo.MongoClient()

In [6]:
for line in tqdm.tqdm(response.iter_lines(), mininterval=1):
    if line:
        tweet = json.loads(line)
        tweet["_timestamp"] = datetime.datetime.now(tz=KST).isoformat()  # timestamp 추가
        mongo.twitter.sample.insert_one(tweet)  # 저장

0it [00:30, ?it/s]


ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 61ffa32d269cb3732d799b14, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused')>]>

In [7]:
tweet

{'data': {'id': '1490271535638188033',
  'text': 'RT @asmaa_but: لا إله إلّا الله وحده لا شريك له، له المُلك وله الحمد، وهو على كل شيء قدير"،\nردد هذا الذكر 100 مرة، فإنه أفضل ما تستقبل به ي…'},
 '_timestamp': '2022-02-06T19:30:23.340540+09:00',
 '_id': ObjectId('61ffa33f269cb3732d799b15')}

In [83]:

# 참고자료: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Sampled-Stream/sampled-stream.py